In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
skip_training = False

In [ ]:
from pathlib import Path
import torch

import grpo_blackjack_agent as a
import grpo_blackjack_train as t
import utils as u

In [ ]:
work_dir = Path().cwd()/'results'

In [ ]:
cfg_args = {
    "exp_name": "grpo_v2",
    "seed": 2025,
    "env_name": "Blackjack-v1",
    "model_name": "Blackjack-v1",
    "max_episode_steps": 100, # not relevant for Blackjack
    "epochs": 2,
    "clip": 0.1,
    "hidden_size": 16,
    "train_episodes": 20000000,
    "batch_size": 2048,
    "min_update_samples": 50000,
    "testing": False,
    "model_path": "default",
    "save_video": False,
    "save_model": True,
    "save_logging": True,
    "silent": False,
    "use_wandb": True,
    "run_suffix": 0,
}

In [ ]:
if not skip_training:
    t.train(cfg_args=cfg_args)

In [ ]:
if not skip_training:
    u.plot_reward(Path().cwd()/'results'/'logging'/'Blackjack-v1_2025.csv', 'Blackjack')

In [ ]:
model_path = work_dir / 'model' / f'{cfg_args["model_name"]}_params.pt'

observation_space_dim = 3
action_space_dim = 2
policy = a.Policy(observation_space_dim, action_space_dim)

print(f"Loading model from {model_path}...")
state_dict = torch.load(model_path, map_location='cuda')
policy.load_state_dict(state_dict)

policy.eval()

In [ ]:
import matplotlib.pyplot as plt

from grpo_utils import create_grids_nn, create_plots

policy_grid_usable_ace = create_grids_nn(policy, usable_ace=True)
fig1 = create_plots(policy_grid_usable_ace, title="With usable ace")
plt.show()

policy_grid_no_ace = create_grids_nn(policy, usable_ace=False)
fig2 = create_plots(policy_grid_no_ace, title="Without usable ace")
plt.show()

In [ ]:
from grpo_utils import create_hit_prob_grid, create_hit_prob_plot
import matplotlib.pyplot as plt

prob_grid_ace = create_hit_prob_grid(policy, usable_ace=True)
create_hit_prob_plot(prob_grid_ace, "Hit prob – usable ace")
plt.show()

prob_grid_no_ace = create_hit_prob_grid(policy, usable_ace=False)
create_hit_prob_plot(prob_grid_no_ace, "Hit prob – no usable ace")
plt.show()